In [9]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [10]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [11]:
device=torch.cuda.get_device_name()
device

'NVIDIA RTX 2000 Ada Generation Laptop GPU'